# Classifying Events with:
* Logistic Regression
* Random Forests
* SVM
* Naive Bayes



In [1]:
#Use mne XDawn for preprocessing

import numpy as np
import matplotlib.pyplot as plt
import matplotlib

from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import MinMaxScaler

import mne
from mne import io, pick_types, read_events, Epochs
# from mne.datasets import sample
from mne.preprocessing import Xdawn
from mne.decoding import Vectorizer
from mne.viz import tight_layout

import os
import os.path as op

matplotlib.rcParams['figure.figsize'] = (15.0, 10.0)
matplotlib.rcParams.update({'font.size': 15})

#data path for each run of each subject.
drive_data_path = 'E:\eeg_data'

#data path on my external hdd for folder containing all tests of each subject in one file
drive_all_data_path = 'E:\eeg_data\ica_140_500_0.1'

In [2]:
# import all runs data from 10 subjects

#initalize a np array of numbers from 1 to 10. This is the number of subjects
numb_subj = np.arange(1,11,1)


#empty array that will have all file names
files = []

for i in range(len(numb_subj)):
    files.append(str('subject' + str(numb_subj[i]) + '_all_runs-epo.fif'))

#convert to np array
files = np.array(files)

all_data = []
for i in range(len(files)):
     all_data.append(mne.read_epochs(op.join(drive_all_data_path, files[i]),
                          preload=True));



Reading E:\eeg_data\ica_140_500_0.1\subject1_all_runs-epo.fif ...
Isotrak not found
    Read a total of 1 projection items:
        Average EEG reference (1 x 61) active
    Found the data of interest:
        t =       0.00 ...    1400.00 ms
        0 CTF compensation matrices available
1121 matching events found
Created an SSP operator (subspace dimension = 1)
1121 matching events found
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Reading E:\eeg_data\ica_140_500_0.1\subject2_all_runs-epo.fif ...
Isotrak not found
    Read a total of 1 projection items:
        Average EEG reference (1 x 61) active
    Found the data of interest:
        t =       0.00 ...    1400.00 ms
        0 CTF compensation matrices available
545 matching events found
Created an SSP operator (subspace dimension = 1)
545 matching events found
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Reading E:\eeg_data\ica_140_500_0.1\subject3_all_runs-epo.fif 

In [3]:
#extract event_id 4 and 5 from all data

all_epochs = []
for i in range(len(all_data)):
    
    all_epochs.append(all_data[i][(all_data[i].events[:,-1] == 4) | (all_data[i].events[:,-1] == 5)])

In [5]:
#concatenate epochs list
epochs = mne.concatenate_epochs(all_epochs)

C:\Users\Esimk_000\Anaconda3\envs\py27\lib\site-packages\mne\epochs.py:2651: RuntimeWarning: overflow encountered in long_scalars
  int((10 + tmax) * epochs.info['sfreq']))


The events passed to the Epochs constructor are not chronologically ordered.
3826 matching events found


<ipython-input-5-35e9a5f5539c>:2: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  epochs = mne.concatenate_epochs(all_epochs)


Created an SSP operator (subspace dimension = 1)
0 bad epochs dropped


In [9]:
epochs = epochs.pick_channels(epochs.ch_names[:60])

In [10]:
clf = make_pipeline(Xdawn(n_components = 3),
                    Vectorizer(),
                    MinMaxScaler(),
                    LogisticRegression(penalty='l1'))
#cross validator
cv = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)


#Do cross-validation
labels = epochs.events[:,-1]
preds = np.empty(len(labels))

for train, test in cv.split(epochs, labels):
    clf.fit(epochs[train], labels[train])
    preds[test] = clf.predict(epochs[test])
    
target_names = ['new', 'scramble']
report = classification_report(labels, preds, target_names=target_names)
print(report)

             precision    recall  f1-score   support

        new       0.66      0.74      0.70      2214
   scramble       0.57      0.47      0.52      1612

avg / total       0.62      0.63      0.62      3826

